Project: Search Engine with Tools and Agents

In [8]:
### Create Tools - ARXIV and Wikipedia
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper


In [9]:
### Use inbuilt tool of Wikipedia
# Initialize the Wikipedia API wrapper with settings for top results and content length
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
# Create a Wikipedia query tool using the API wrapper
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name  # Display the name of the Wikipedia tool

'wikipedia'

In [10]:
### Use the inbuilt tool of Arxiv
# Initialize the Arxiv API wrapper with settings for top results and content length
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
# Create an Arxiv query tool using the API wrapper
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name  # Display the name of the Arxiv tool

'arxiv'

In [11]:
### Combine both the tools
# Combine the Arxiv and Wikipedia tools into a list for use in agents
tools = [arxiv, wiki]

In [12]:
### Custom tools (RAG)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from a .env file

openai_api_key = os.getenv("OPENAI_API_KEY")  # Retrieve OpenAI API key from environment variables


In [14]:
# Initialize OpenAI embeddings with the API key
embeddings = OpenAIEmbeddings(api_key=openai_api_key)
# Load documents from a web source
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
# Split documents into chunks for processing
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
# Create a FAISS vector store from the documents and embeddings
vectordb = FAISS.from_documents(documents, embeddings)
# Create a retriever from the vector store
retriever = vectordb.as_retriever()
retriever  # Display the retriever object

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025FDCE2E890>, search_kwargs={})

In [15]:
### Create retriever tool
from langchain.tools.retriever import create_retriever_tool
# Create a retriever tool for searching information about Langsmith
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name  # Display the name of the retriever tool

'langsmith-search'

In [16]:
# Add the retriever tool to the list of tools
tools = [arxiv, wiki, retriever_tool]

In [17]:
tools  # Display the list of tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025FA0061760>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025FDCE2E890>, search_kwargs={}), docum

In [20]:
## Tools, LLM --> AgentExecutor
# Model Initialization
from langchain_groq import ChatGroq  # Import the ChatGroq class from langchain_groq
groq_api_key = os.getenv("GROQ_API_KEY")  # Retrieve Groq API key from environment variables
# Initialize the language model with the specified model and API key
model = ChatGroq(model="Llama3-8b-8192", groq_api_key=groq_api_key)

In [21]:
### Prompt Template
from langchain import hub
# Pull a prompt template from the Langchain hub
prompt = hub.pull("hwchase17/openai-functions-agent")  # Reference to the prompt template | #https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/
prompt.messages  # Display the messages in the prompt template

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
### Agents
from langchain.agents import create_openai_tools_agent
# Create an agent using the model, tools, and prompt
agent = create_openai_tools_agent(model, tools, prompt)
agent  # Display the agent object

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [24]:
## Agent Executor
from langchain.agents import AgentExecutor
# Create an agent executor to run the agent with the tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor  # Display the agent executor object

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
# Invoke the agent executor with a query about Langsmith
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation neede

{'input': 'Tell me about Langsmith',
 'output': 'Thank you for using the tool. The output from the tool call id "call_vj7q" is the documentation page for LangSmith, a platform for building production-grade LLM applications. It appears to provide a range of features and tools for developers to create, test, and deploy their own LLM applications.'}

In [27]:
# Invoke the agent executor with a query about Machine Learning
agent_executor.invoke({"input": "What is Machine Learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith-search` with `{'query': 'Machine learning optimization'}`


Trace LLM Applications: Gain visibility into LLM calls and other parts of your application's logic.
Evaluate Performance: Compare results across models, prompts, and architectures to identify what works best.
Improve Prompts: Quickly refine prompts to achiev

{'input': 'What is Machine Learning?',
 'output': "This seems to be a lot of information about LangSmith, a tool for evaluating and improving AI models. Since we're not looking for information about LangSmith specifically, I'll stop here."}